In [1]:
# NUMPY & PYTORCH COMPATIBILITY FIX - Run this cell first

# Fix NumPy version compatibility
!pip install "numpy<2.0" --force-reinstall

# Fix PyTorch versions
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# Install other required packages
!pip install timm einops opencv-python pillow tensorboard

print("✅ All packages fixed!")
print("🔄 RESTART RUNTIME NOW: Runtime → Restart Runtime")
print("Then run your main code.")

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires torch>=1.13.0, which is not installed.
sentence-transformers 4.1.0 requires torch>=1.11.0, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
accelerate 1.7.0 requires torch>=2.0.0, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 706.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=5db6d20d356340225bf555c72f090bf9e7f02afb7da9fd49553057a05d45da46
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
✅ All packages fixed!
🔄 RESTART RUNTIME NOW: Runtime → Restart Runtime


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd RVRT

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
import gc
import numpy as np
from pathlib import Path
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Memory optimization settings
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

class ImagePairDataset(Dataset):
    def __init__(self, blur_dir, sharp_dir, img_size=256, max_samples=None):
        self.blur_dir = Path(blur_dir)
        self.sharp_dir = Path(sharp_dir)
        self.img_size = img_size

        # Get matching pairs
        blur_files = sorted([f for f in os.listdir(blur_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        sharp_files = sorted([f for f in os.listdir(sharp_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        # Match files by name
        self.pairs = []
        for blur_file in blur_files:
            if blur_file in sharp_files:
                self.pairs.append((blur_file, blur_file))

        # Limit dataset size for memory management
        if max_samples and len(self.pairs) > max_samples:
            self.pairs = self.pairs[:max_samples]

        print(f"Found {len(self.pairs)} image pairs")

        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        blur_name, sharp_name = self.pairs[idx]

        # Load images
        blur_img = Image.open(self.blur_dir / blur_name).convert('RGB')
        sharp_img = Image.open(self.sharp_dir / sharp_name).convert('RGB')

        # Apply transforms
        blur_tensor = self.transform(blur_img)
        sharp_tensor = self.transform(sharp_img)

        return blur_tensor, sharp_tensor

# Simplified RVRT-inspired model for efficiency
class SimpleRVRT(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_feat=64, num_blocks=6):
        super(SimpleRVRT, self).__init__()

        # Feature extraction
        self.conv_first = nn.Conv2d(in_channels, num_feat, 3, 1, 1)

        # Residual blocks
        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(num_feat, num_feat, 3, 1, 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(num_feat, num_feat, 3, 1, 1)
            ) for _ in range(num_blocks)
        ])

        # Output layer
        self.conv_last = nn.Conv2d(num_feat, out_channels, 3, 1, 1)

    def forward(self, x):
        feat = self.conv_first(x)

        for block in self.blocks:
            feat = feat + block(feat)  # Residual connection

        out = self.conv_last(feat)
        return out

# Training configuration
class Config:
    # Paths - UPDATE THESE WITH YOUR ACTUAL PATHS
    blur_dir = "/content/drive/MyDrive/blur"  # Update this path
    sharp_dir = "/content/drive/MyDrive/sharp"  # Update this path

    # Training parameters (optimized for Colab)
    batch_size = 4  # Small batch size to prevent OOM
    learning_rate = 1e-4
    num_epochs = 50
    img_size = 256
    max_samples = 2000  # Limit dataset size

    # Model parameters
    num_feat = 32  # Reduced from typical 64
    num_blocks = 4  # Reduced from typical 16

    # Checkpoint
    checkpoint_dir = "/content/checkpoints"
    save_every = 10

def train_model():
    # Setup
    os.makedirs(Config.checkpoint_dir, exist_ok=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Dataset and DataLoader
    print("Loading dataset...")
    dataset = ImagePairDataset(
        Config.blur_dir,
        Config.sharp_dir,
        Config.img_size,
        Config.max_samples
    )

    dataloader = DataLoader(
        dataset,
        batch_size=Config.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    # Model
    print("Initializing model...")
    model = SimpleRVRT(
        num_feat=Config.num_feat,
        num_blocks=Config.num_blocks
    ).to(device)

    # Loss and optimizer
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=Config.learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    # Training loop
    print("Starting training...")
    model.train()

    for epoch in range(Config.num_epochs):
        epoch_loss = 0.0

        # Progress bar
        pbar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{Config.num_epochs}')

        for batch_idx, (blur_imgs, sharp_imgs) in enumerate(pbar):
            blur_imgs = blur_imgs.to(device, non_blocking=True)
            sharp_imgs = sharp_imgs.to(device, non_blocking=True)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(blur_imgs)
            loss = criterion(outputs, sharp_imgs)

            # Backward pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{epoch_loss/(batch_idx+1):.4f}'
            })

            # Memory cleanup every 50 batches
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        # Scheduler step
        scheduler.step()

        # Save checkpoint
        if (epoch + 1) % Config.save_every == 0:
            checkpoint_path = f"{Config.checkpoint_dir}/model_epoch_{epoch+1}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss / len(dataloader),
            }, checkpoint_path)
            print(f"Checkpoint saved: {checkpoint_path}")

        print(f"Epoch {epoch+1} completed. Average Loss: {epoch_loss/len(dataloader):.4f}")

        # Memory cleanup
        torch.cuda.empty_cache()
        gc.collect()

    # Save final model
    final_path = f"{Config.checkpoint_dir}/final_model.pth"
    torch.save(model.state_dict(), final_path)
    print(f"Training completed! Final model saved: {final_path}")

# Test function
def test_model(model_path, test_blur_path, output_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load model
    model = SimpleRVRT(num_feat=Config.num_feat, num_blocks=Config.num_blocks)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Load and process test image
    transform = transforms.Compose([
        transforms.Resize((Config.img_size, Config.img_size)),
        transforms.ToTensor(),
    ])

    blur_img = Image.open(test_blur_path).convert('RGB')
    blur_tensor = transform(blur_img).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        sharp_tensor = model(blur_tensor)

    # Convert back to image
    sharp_img = transforms.ToPILImage()(sharp_tensor.squeeze(0).cpu())
    sharp_img.save(output_path)
    print(f"Deblurred image saved: {output_path}")

# Mount Google Drive (if using Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# Main execution
if __name__ == "__main__":
    print("RVRT Fine-tuning for Image Deblurring")
    print("=" * 50)

    # Check GPU
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # Update paths in Config class above before running
    print("\n⚠️  IMPORTANT: Update blur_dir and sharp_dir paths in Config class!")
    print("Example:")
    print("Config.blur_dir = '/content/drive/MyDrive/your_dataset/blur'")
    print("Config.sharp_dir = '/content/drive/MyDrive/your_dataset/sharp'")

    # Uncomment to start training
    train_model()

    # Uncomment to test model
    # test_model(
    #     "/content/checkpoints/final_model.pth",
    #     "/path/to/test/blur/image.jpg",
    #     "/content/deblurred_output.jpg"
    # )

[Errno 2] No such file or directory: 'RVRT'
/content
RVRT Fine-tuning for Image Deblurring
GPU: Tesla T4
GPU Memory: 15.8 GB

⚠️  IMPORTANT: Update blur_dir and sharp_dir paths in Config class!
Example:
Config.blur_dir = '/content/drive/MyDrive/your_dataset/blur'
Config.sharp_dir = '/content/drive/MyDrive/your_dataset/sharp'
Using device: cuda
Loading dataset...
Found 1029 image pairs
Initializing model...
Starting training...


Epoch 1/50: 100%|██████████| 258/258 [05:57<00:00,  1.39s/it, Loss=0.1147, Avg Loss=0.1544]


Epoch 1 completed. Average Loss: 0.1544


Epoch 2/50: 100%|██████████| 258/258 [01:08<00:00,  3.78it/s, Loss=0.1437, Avg Loss=0.1331]


Epoch 2 completed. Average Loss: 0.1331


...

Epoch 48 completed. Average Loss: 0.1203


Epoch 49/50: 100%|██████████| 258/258 [01:08<00:00,  3.77it/s, Loss=0.1021, Avg Loss=0.1203]


Epoch 49 completed. Average Loss: 0.1203


Epoch 50/50: 100%|██████████| 258/258 [01:08<00:00,  3.76it/s, Loss=0.1557, Avg Loss=0.1204]

Checkpoint saved: /content/checkpoints/model_epoch_50.pth
Epoch 50 completed. Average Loss: 0.1204
Training completed! Final model saved: /content/checkpoints/final_model.pth


In [3]:
# SIMPLE VIDEO DEBLURRING - Add this in a new cell after training

# Input your video path here
INPUT_VIDEO = "/content/WhatsApp Video 2025-06-12 at 14.10.17_c5f5aa48.mp4" # Change this path
OUTPUT_VIDEO = "/content/deblurred_output.mp4"

# Load trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRVRT(num_feat=Config.num_feat, num_blocks=Config.num_blocks).to(device)
model.load_state_dict(torch.load("/content/checkpoints/final_model.pth"))
model.eval()

# Process video
cap = cv2.VideoCapture(INPUT_VIDEO)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

print("Processing video...")
with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret: break

        # Process frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_tensor = transform(frame_rgb).unsqueeze(0).to(device)
        output_tensor = model(input_tensor)

        # Convert back
        output_pil = transforms.ToPILImage()(output_tensor.squeeze(0).cpu())
        output_pil = output_pil.resize((width, height))
        output_frame = cv2.cvtColor(np.array(output_pil), cv2.COLOR_RGB2BGR)

        out.write(output_frame)

cap.release()
out.release()
print(f"✅ Deblurred video saved: {OUTPUT_VIDEO}")

Processing video...
✅ Deblurred video saved: /content/deblurred_output.mp4
